In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import f_oneway
from scipy.spatial.distance import cdist
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import heapq as hq

In [ ]:
# 计算方差窗口的大小
def computeWinSize(length, b):
    if 0 < length <= b:
        return 1
    else:
        return length // b

In [ ]:
# 计算标准差数组
def computeStdArr(series, b):
    l = len(series)
    # 窗口下取整
    w = computeWinSize(l, b)
    # print(w)
    res = []
    for i in range(w, l - w):
        win_arr = [0 for i in range(2 * w + 1)]
        cnt = 0
        for j in range(i - w, w + i + 1):
            win_arr[cnt] = series[j]
            cnt += 1
        std = np.std(win_arr, ddof=0)
        res.append(std)
    return res

In [ ]:
# 选择L*α最大的点的下标,，保留下标
def selectLargePoint(series, a):
    l = len(series)
    # 生成l*a个
    res = hq.nlargest(int(l * a), range(l), key=lambda x: series[x])
    return res

In [ ]:
# 判断是否为波峰波谷
def isPeakOrValley(std_arr, index, b):
    win_size = computeWinSize(len(std_arr), b)
    flag = False
    if win_size == 1:
        if ((std_arr[index] > std_arr[index - 1] and std_arr[index] > std_arr[index + 1]) or (
                std_arr[index] < std_arr[index - 1] and std_arr[index] < std_arr[index + 1])):
            flag = True
    elif win_size == 2:
        if ((std_arr[index] >= std_arr[index - 1] and std_arr[index] > std_arr[index - 2] and std_arr[index] >= std_arr[
            index + 1] and std_arr[index] > std_arr[index + 2])
                or
                (std_arr[index] <= std_arr[index - 1] and std_arr[index] < std_arr[index - 2] and std_arr[index] <=
                 std_arr[
                     index + 1]) and std_arr[index] < std_arr[index + 2]):
            flag = True
    elif win_size == 3:
        if ((std_arr[index] >= std_arr[index - 1] and std_arr[index] > std_arr[index - 2] and std_arr[index] > std_arr[
            index - 3]
             and std_arr[index] >= std_arr[index + 1] and std_arr[index] > std_arr[index + 2] and std_arr[index] >
             std_arr[
                 index + 3])
                or
                (std_arr[index] <= std_arr[index - 1] and std_arr[index] < std_arr[index - 2] and std_arr[index] <
                 std_arr[index - 3]) and std_arr[index] <= std_arr[index + 1] and std_arr[index] < std_arr[
                    index + 2] and std_arr[index] < std_arr[index + 3]):
            flag = True
    elif win_size == 4:
        if ((std_arr[index] >= std_arr[index - 1] and std_arr[index] > std_arr[index - 2] and std_arr[index] > std_arr[
            index - 3] and std_arr[index] > std_arr[index - 4]
             and std_arr[index] >= std_arr[index + 1] and std_arr[index] > std_arr[index + 2] and std_arr[index] >
             std_arr[
                 index + 3] and std_arr[index] > std_arr[index + 4])
                or
                (std_arr[index] <= std_arr[index - 1] and std_arr[index] < std_arr[index - 2] and std_arr[index] <
                 std_arr[
                     index - 3] and std_arr[index] < std_arr[index - 4]
                 and std_arr[index] <= std_arr[index + 1] and std_arr[index] < std_arr[index + 2] and std_arr[index] <
                 std_arr[index + 3] and std_arr[index] < std_arr[index + 4])):
            flag = True
    else:
        if ((std_arr[index] > std_arr[index - 1] and std_arr[index] > std_arr[index - 2] and std_arr[index] > std_arr[
            index - 3] and std_arr[index] > std_arr[index - 4] and std_arr[index] > std_arr[index - 5]
             and std_arr[index] > std_arr[index + 1] and std_arr[index] > std_arr[index + 2] and std_arr[index] >
             std_arr[
                 index + 3] and std_arr[index] > std_arr[index + 4] and std_arr[index] > std_arr[index + 5])
                or
                (std_arr[index] < std_arr[index - 1] and std_arr[index] < std_arr[index - 2] and std_arr[index] <
                 std_arr[
                     index - 3] and std_arr[index] < std_arr[index - 4] and std_arr[index] < std_arr[index - 5]
                 and std_arr[index] < std_arr[index + 1] and std_arr[index] < std_arr[index + 2] and std_arr[index] <
                 std_arr[
                     index + 3] and std_arr[index] < std_arr[index + 4] and std_arr[index] < std_arr[index + 5])):
            flag = True
    return flag



In [ ]:
# 找到所有合适的区间
def fitPoint(stdA, indexA, b):
    w = computeWinSize(len(stdA), b)
    l = len(indexA)
    res = []
    for i in range(0, l):
        val = indexA[i]
        if w <= val < l - w:
            if isPeakOrValley(stdA, val, b):
                res.append(val + 2 * w + 1)
    return res

In [ ]:
# 提取特征关键点
def extractFeaturePoint(arr, a, b):
    stdArrOne = computeStdArr(arr, b)
    stdArrTwo = computeStdArr(stdArrOne, b)
    largePoint = selectLargePoint(stdArrTwo, a)
    indexArr = fitPoint(stdArrTwo, largePoint, b)
    return indexArr

In [ ]:
# def findDistances(S,Wl):
#     Ds = []
#     Indx = []
#     for Wil in Wl:
#         dist = cdist(np.array([S]),np.vstack(Wil),metric='seuclidean')
#         minDist = dist.min()
#         idx = np.where(dist,minDist)
#         Ds.append(minDist)
#         Indx.append(idx)
#     return np.array(Ds),np.array(Indx)
def findDistances(S,Wl):
    Ds = []
    for Wil in Wl:
        Ds.append(cdist(np.array([S]),np.vstack(Wil),metric='seuclidean').min())
    return np.array(Ds)

In [ ]:
def assessCandidate(Ds,C):
    class_groups = []
    for c in np.unique(C):
        class_groups.append(Ds[C==c].tolist())
#  返回f值
    return f_oneway(*class_groups).statistic

In [ ]:
def sortByQuality(shapelets):
    return sorted(shapelets, key=lambda tup: tup[1],reverse=True)

In [ ]:
def removeSelfSimilar(shapelets):
    queue = shapelets[:]
    df = pd.DataFrame(shapelets)

    keep_df = pd.DataFrame()

    while len(df) != 0:
        pop_item = queue.pop(0)
        s,f,interval,k = pop_item

        keep_df = pd.concat((keep_df,pd.DataFrame([pop_item])))

        df = df[~df[2].apply(lambda x: interval.overlaps(x))]
        queue = df.values.tolist()

    return keep_df.drop(2,axis=1).values.tolist()

In [ ]:
def merge(k,kShapelets,shapelets):
    total_shapelets = kShapelets + shapelets
    return sortByQuality(total_shapelets)[:k]

In [ ]:
def generateCandidates(Ti,l,keyPoint,ylt):
    # ylt为松弛参数
    candidate = []
    param = []
    if l % 2 == 0:
        l = l // 2
    else:
        l = l // 2 + 1
    for i in keyPoint:
        # candidate keypoint
        for j in range(ylt):
            # 从中心点两侧的松弛长度进行计算
            # 包含区间，keypoint，起点，终点
            candidate.append(np.array(Ti[max(i - ylt - l,0) : min(i + ylt + l + 1,len(Ti))]))
            param.append([i,max(i - ylt - l,0),min(i + ylt + l + 1,len(Ti))])
    return candidate,param

In [ ]:
def create_Wl(T,l,keyPoints,sample_nums,ylt):
    res = []
    for sn in range(sample_nums):
        cd,pa = generateCandidates(T[sn],l,keyPoints[sn],ylt)
        res.append([cd,pa])
    return res

In [71]:
def SelectShapelet(data,label,lmin,lmax,alp,blt,ylt):
    # 原始data顺序 样本数 维度数 序列长度
    data = np.transpose(data,axes=(1,0,2))
    # 现在data顺序 维度数 样本数 序列长度
    dim_nums = data.shape[0]
    sample_nums = data.shape[1]
    for dn in tqdm(range(dim_nums)):
        T = data[dn]
        keyPoints = []
        for sm in range(sample_nums):
            keyPoints.append(extractFeaturePoint(data[dn][sm],alp,blt))
        # print(keyPoints)
        precompute_Wl = {l : create_Wl(T,l,keyPoints,sample_nums,ylt) for l in np.arange(lmin,lmax+1)}

        kShapelets = []
        for i,Ti in enumerate(T):
            shapelets = []
            for l in np.arange(lmin,lmax+1):
                # S为所有的长度
                Al = precompute_Wl[l]
                Wl,Pl = Al[0],Al[1]
                for index,S in enumerate(Wl[i]):
                     # 和所有样本上的候选者比较
                    #  S 区间，keypoint，松弛长度
                    # Ds,Indx = findDistances(S,Wl)

                    Ds = findDistances(S,Wl)
                    quality = assessCandidate(Ds,label)
                    #  区间值 度量  区间 关键点
                    shapelets.append((S,quality,pd.Interval(Pl[index][1],Pl[index][2]-1,closed='both'),Pl[index][0]))
            shapelets = sortByQuality(shapelets)
            shapelets = removeSelfSimilar(shapelets)
            kShapelets = merge(k,kShapelets,shapelets)
        print(kShapelets)









In [ ]:
def load_raw_ts(path, dataset):
    path = path + "raw//" + dataset + "//"
    # 训练集
    x_train = np.load(path + 'X_train.npy')
    x_train = np.transpose(x_train, axes=(0, 2, 1))
    x_test = np.load(path + 'X_test.npy')
    x_test = np.transpose(x_test, axes=(0, 2, 1))
    y_train = np.load(path + 'y_train.npy')
    y_test = np.load(path + 'y_test.npy')
    labels = np.concatenate((y_train, y_test), axis=0)
    nclass = int(np.amax(labels)) + 1
    return x_train, x_test, y_train.reshape(-1), y_test.reshape(-1), nclass

In [73]:
# import os
# folder = 'ItalyPowerDemand'
# train_df = pd.read_csv(os.path.join(folder,'ItalyPowerDemand_TRAIN.txt'),sep="  ",header=None,error_bad_lines=False)
# print(train_df.shape)
# test_df = pd.read_csv(os.path.join(folder,'ItalyPowerDemand_TEST.txt'),sep="  ",header=None,error_bad_lines=False)
# print(test_df.shape)
# C= train_df.pop(0)
# T = [np.array(Ti) for Ti in train_df.values.tolist()]
# C_test = test_df.pop(0)
# T_test = [np.array(Ti) for Ti in test_df.values.tolist()]

# folder = 'SonyAIBORobotSurface2'
# train_df = pd.read_csv(os.path.join(folder,'SonyAIBORobotSurface2_TRAIN.txt'),sep="  ",header=None,error_bad_lines=False)
# print(train_df.shape)
# test_df = pd.read_csv(os.path.join(folder,'SonyAIBORobotSurface2_TEST.txt'),sep="  ",header=None,error_bad_lines=False)
# print(test_df.shape)
# C= train_df.pop(0)
# T = [np.array(Ti) for Ti in train_df.values.tolist()]
# C_test = test_df.pop(0)
# T_test = [np.array(Ti) for Ti in test_df.values.tolist()]
# print(type(C),type(T))
# C 为标签 T 为序列
# target_train 为标签 data_train 为序列
data_train, data_test, target_train, target_test, nclass = load_raw_ts("D://tmppro//data//", "StandWalkJump")
sample_nums = data_train.shape[0]
dim_nums = data_train.shape[1]
series_length = data_train.shape[2]
# print(data_train.shape)
# for sn in range(sample_nums):
#     for dn in range(1):
#         plt.plot(data_train[sn][dn])
# plt.title("777")

In [74]:
import random

# # 这里的N指的是序列的样本数量
# # N = len(T)
# # N_test = len(T_test)
# # print(N,N_test)
# # N=200
# #N_test = 5000
lmin = 5
lmax = series_length//2
# 选取shapelet的数量
k = 50
alp = random.choice([0.3, 0.4])
blt = random.choice([100, 90, 80, 70, 60, 50])
ylt = random.choice([5,6,7,8])
#
# # min_seq_len = 20
# # max_seq_len = 30
# # T,C = generate_dataset(N,min_seq_len,max_seq_len)
# # T_test,C_test = generate_dataset(N_test,min_seq_len,max_seq_len)

In [ ]:
kshapelets = SelectShapelet(data_train,target_train,lmin,lmax,alp,blt,ylt)

  0%|          | 0/4 [05:44<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
# kshapelets = ShapeletCachedSelection(T,lmin,lmax,k)

In [ ]:
# for Ti in T:
#     plt.plot(Ti)
# plt.title("Training Trajectories")

In [ ]:
# for shape in kshapelets:
#     plt.plot(shape[0])
# plt.title("Discriminatory Shapelets")
# plt.show()

In [ ]:
# feature_shaplets,quality_scores = zip(*kshapelets)

In [ ]:
# dataset = np.zeros((N,k))
# for i,feature in enumerate(tqdm(feature_shaplets)):
#     l = len(feature)
#     Wl = create_Wl(T,l)
#     Ds = findDistances(feature,Wl)
#     dataset[:,i] = Ds
# # print(dataset)

In [ ]:
# dataset_test = np.zeros((N_test,k))
# for i,feature in enumerate(tqdm(feature_shaplets)):
#     l = len(feature)
#     Wl = create_Wl(T_test,l)
#     Ds = findDistances(feature,Wl)
#     dataset_test[:,i] = Ds

In [ ]:
# from sklearn.linear_model import LogisticRegressionCV
# from sklearn.svm import LinearSVC
# from sklearn.preprocessing import StandardScaler
# from sklearn.metrics import classification_report
# from sklearn.metrics import accuracy_score

In [ ]:
# sc = StandardScaler()
# # model = LogisticRegressionCV().fit(sc.fit_transform(dataset),C)
# model = LinearSVC().fit(sc.fit_transform(dataset),C)

In [ ]:
# print(classification_report(C,model.predict(sc.transform(dataset))))

In [ ]:
# print(classification_report(C_test,model.predict(sc.transform(dataset_test))))
# print(accuracy_score(C_test,model.predict(sc.transform(dataset_test))))